# Model Setup
Model 1 takes in img outputs moving probability
Model 2 takes in a sequence of 16 moving probs outputs falling probability

In [ ]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

import cv2

GPU limitation

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

# Building the Models

## Model 1

### Train

In [ ]:
data = DataLoader.from_folder(r"./datasets/model1_data")
train_data, test_data = data.split(0.9)

In [ ]:
model = image_classifier.create(train_data)

### Evaluate & Export

In [ ]:
loss, accuracy = model.evaluate(test_data)

In [ ]:
model.export(export_dir='.')

## Model 2

### Train

In [ ]:
default_data = np.loadtxt('./datasets/model2_data/default.txt')
falling_data = np.loadtxt('./datasets/model2_data/falling.txt')

In [ ]:
inputs = np.concatenate((default_data, falling_data))
outputs = np.concatenate((np.zeros(len(default_data)), np.ones(len(falling_data))))

# Convert inputs and outputs to TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))
dataset = dataset.shuffle(len(inputs)).batch(128)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(16,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(dataset, epochs=6)

### Evaluate & Export

# Predict

## Reformat Input

In [ ]:
def resize_video(input_path, output_path, width, height):
  # Open the video file
  video = cv2.VideoCapture(input_path)

  # Get the original video's width and height
  original_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
  original_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  
  # Create a VideoWriter object to save the resized video
  fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Codec for the output video
  fps = video.get(cv2.CAP_PROP_FPS)
  writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  
  while True:
    # Read a frame from the original video
    ret, frame = video.read()
    if not ret: break
    # Resize the frame to the desired width and height
    resized_frame = cv2.resize(frame, (width, height))
    # Write the resized frame to the output video file
    writer.write(resized_frame)

# Release the video capture and writer objects
  video.release()
  writer.release()

In [ ]:
# Resize vid
video_path = "s-walk2.mp4"
output_path = "resized_IMG1616_falling.mp4"
target_width = 224
target_height = 224

#resize_video(video_path, output_path, target_width, target_height)

# Load resized vid
cap = cv2.VideoCapture(output_path)
# frame_rate = 30
# cap.set(cv2.CAP_PROP_FPS, frame_rate) DOESN'T WORK
print(cap.get(cv2.CAP_PROP_FPS))

## Model 1

In [ ]:
# Initialize list to store the frame classifications
frame_classifications = []
moving_prob = []
# Loop through the frames of the video (need to change to 30 fps)
while True:
    ret, frame = cap.read() 
    #just need to figure out if this is 30 fps

    if not ret: # Break the loop if the video has ended
        break

    ''' Formulate Input Data (frame_rgb) '''
    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Make frame input data and ensure its type matches the model
    frame_rgb = np.expand_dims(frame_rgb, axis=0)

    ''' Classify the Frame '''
    interpreter = tf.lite.Interpreter(model_path="model.tflite")
    interpreter.allocate_tensors()

    # get_output_details() and get_input_details() return list of dictionaries of tensor details
    # keys: name, index, shape, shape_signature, dtype, quantization, ...
    # len(input) = len(output) = 1, so access the first element
    output = interpreter.get_output_details()
    input = interpreter.get_input_details()
    output_index = output[0]['index']
    input_index = input[0]['index']

    # set input -> invoke -> access output
    interpreter.set_tensor(input_index, frame_rgb)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_index)
    # If the output_data shape is (batch_size, num_classes), select the first frame
    output_data = output_data[0]

    # Convert each entry into probability
    output_probs = tf.nn.softmax(output_data.astype(float))

    # Find the index of the highest probability
    predicted_index = np.argmax(output_data)

    # Assuming you have a list of class labels corresponding to the model's output classes
    class_labels = ["Moving", "Still"]

    # Get the predicted class label
    predicted_class = class_labels[predicted_index]

    # Print the predicted class label
    # print("Predicted Class:", predicted_class)
    frame_classifications.append((predicted_class, max(output_probs.numpy())))
    
    prob = np.around(max(output_probs.numpy()), decimals = 2)
    if(predicted_class == "Still"):
        
        moving_prob.append(np.subtract(1, prob))
    else:
        moving_prob.append(prob)

In [ ]:
for i, element in enumerate(moving_prob):
    print(element, end=' ')
    if (i + 1) % 16 == 0:
        print()

In [ ]:
#